CatBoost Regression Modeling

In [2]:
import pickle
import pathlib
import numpy as np
import pandas as pd
import catboost

In [3]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)
clean_data_path = DATA_DIR / 'processed' / 'ames_clean_eng.pkl'
with open(clean_data_path, 'rb') as file:
    model_data = pickle.load(file)

c:\Users\Pedro Antônio Silva\Desktop\Semestre 4\Machine Learning\proj_1\ames\data


ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'

In [ ]:
y = model_data['SalePrice'].copy()
X = model_data.drop(columns=['SalePrice']).copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from catboost import CatBoostRegressor

# CatBoost model setup
model = CatBoostRegressor(verbose=0)
model.fit(X_train, y_train)

# Predictions on test data
ypred = model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(y_test, ypred))
print(f'RMSE: {RMSE}')
error_percentage = 100 * (10**RMSE - 1)
print(f'Average error is {error_percentage:.2f}%')

RMSE: 0.04596950899383926
Average error is 11.17%


In [ ]:
# Hyperparameter tuning through grid search
from sklearn.model_selection import GridSearchCV

# CatBoost model setup
model = CatBoostRegressor(verbose=0)

# best params that were already tuned = {
# 'bagging_temperature': 0,
# 'border_count': 32,
# 'depth': 6,
# 'iterations': 500,
# 'l2_leaf_reg': 1,
# 'learning_rate': 0.05,
# 'random_strength': 1}

param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 500],
    'l2_leaf_reg': [1, 5, 9],
    'border_count': [32, 128],
    'bagging_temperature': [0, 5],
    'random_strength': [0.5, 1, 2]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0)
grid_search.fit(X_train, y_train)

best_params_catboost = grid_search.best_params_
print(best_params_catboost)
best_model_catboost = grid_search.best_estimator_

{'bagging_temperature': 0, 'border_count': 32, 'depth': 6, 'iterations': 500, 'l2_leaf_reg': 1, 'learning_rate': 0.05, 'random_strength': 1}


In [ ]:
y_pred_catboost = best_model_catboost.predict(X_test)
RMSE_catboost = np.sqrt(mean_squared_error(y_test, y_pred_catboost))
print(f'RMSE: {RMSE_catboost}')
error_percentage_catboost = 100 * (10**RMSE_catboost - 1)
print(f'Average error is {error_percentage_catboost:.2f}%')

RMSE: 0.046184052293911766
Average error is 11.22%
